In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

NUM_OF_ROWS = 25000
ATTACK_NAME = 'PortScan'

---

In [42]:
# import the attack sample dataset
portSamples = pd.read_csv('port_samples.csv')
portSamples

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,1,0.000000,0,0,0.000000,0.000000,0.000000,0.0,0,0.0,...,0.000000,0,1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,97.800000,0,465,97.800000,183.835144,33795.360000,465.0,465,232.5,...,0.000000,0,5,0,0.158153,31.614944,0.158153,0.157461,0.039538,0.068083
2,1,174.000000,174,174,174.000000,0.000000,0.000000,3480.0,174,174.0,...,183.157895,0,0,0,38.026166,0.525954,38.026166,2.005794,2.001377,0.002239
3,1,0.500000,0,1,0.500000,0.500000,0.250000,0.0,0,0.0,...,0.000000,0,2,0,0.072126,27.729286,0.072126,0.072126,0.072126,0.000000
4,1320,0.000000,0,0,0.000000,0.000000,0.000000,0.0,0,0.0,...,0.000000,2632,0,0,38.432056,68.484496,38.432056,1.140289,0.014607,0.053151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,1,76.521739,0,110,76.521739,50.614373,2561.814745,880.0,110,110.0,...,125.714286,0,23,0,35.155495,0.654236,35.155495,4.972319,1.597977,2.295126
749,1,0.500000,0,1,0.500000,0.500000,0.250000,0.0,0,0.0,...,0.000000,0,2,0,0.068069,29.381959,0.068069,0.068069,0.068069,0.000000
750,1,0.500000,0,1,0.500000,0.500000,0.250000,0.0,0,0.0,...,0.000000,0,2,0,0.003770,530.488079,0.003770,0.003770,0.003770,0.000000
751,1,0.500000,0,1,0.500000,0.500000,0.250000,0.0,0,0.0,...,0.000000,0,2,0,0.059732,33.482779,0.059732,0.059732,0.059732,0.000000


In [43]:
# get all the relevant attack rows from the attack sample dataset
portSamples = portSamples[portSamples['Number of Ports'] >= 200]

In [44]:
# print some general information about the attack samples
print(f'Dataset Shape: {portSamples.shape}')
portSamples

Dataset Shape: (31, 26)


,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
4,1320,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,2632,0,0,38.432056,68.484496,38.432056,1.140289,0.014607,0.053151
32,1350,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,2692,0,0,37.986282,70.867688,37.986282,1.325433,0.014116,0.052220
49,1258,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,2509,0,0,39.694650,63.207510,39.694650,1.242401,0.015827,0.057784
64,1370,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,2721,0,0,39.157410,69.488763,39.157410,0.581770,0.014396,0.048303
75,1470,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,2932,0,0,39.450596,74.320804,39.450596,1.110491,0.013460,0.045942
94,1440,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,2862,0,0,39.793435,71.921411,39.793435,0.637111,0.013909,0.044932
119,1322,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,2642,0,0,39.665320,66.607303,39.665320,1.122268,0.015019,0.053150
129,1441,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,2876,0,0,39.821065,72.223081,39.821065,1.155118,0.013851,0.049192
144,1492,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,2972,0,0,39.391055,75.448601,39.391055,0.404771,0.013259,0.041277
149,1320,0.0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,2633,0,0,39.871239,66.037577,39.871239,1.137338,0.015149,0.054810


In [45]:
# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = portSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values
columnsToGather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'SYN Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Total',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

In [46]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
MinMaxDict = {col: (portSamples[col].min() * 0.85, portSamples[col].max() * 1.15) for col in columnsToGather}
MinMaxDict

{'Number of Ports': (198.9, 1715.8),
 'SYN Flag Count': (397.8, 3417.7999999999997),
 'Flow Duration': (6.676324236392975, 45.85192478895187),
 'Packets Per Second': (40.935130011631166, 86.76589115741946),
 'IAT Total': (6.676324236392975, 45.85192478895187),
 'IAT Max': (0.29744905233383173, 6.993011915683747),
 'IAT Mean': (0.011269739764742165, 0.0238991610003671),
 'IAT Std': (0.03508524686586984, 0.21554947266835267)}

In [47]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['Number of Ports', 'SYN Flag Count']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (198, 1715),
 'SYN Flag Count': (397, 3417),
 'Flow Duration': (6.676324236392975, 45.85192478895187),
 'Packets Per Second': (40.935130011631166, 86.76589115741946),
 'IAT Total': (6.676324236392975, 45.85192478895187),
 'IAT Max': (0.29744905233383173, 6.993011915683747),
 'IAT Mean': (0.011269739764742165, 0.0238991610003671),
 'IAT Std': (0.03508524686586984, 0.21554947266835267)}

In [48]:
# creating an empty dataframe before adding values to it
portDataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(portSamples.columns))), columns=portSamples.columns)
portDataset.head(3)

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in portSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    portDataset[col] = int(0)
zeroColumns

['Average Packet Size',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Mean',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Bwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'ACK Flag Count',
 'RST Flag Count']

In [50]:
portDataset.head(3)

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
0,0.0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0,0,0,0,0,0,0,0,0,...,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data

independent_col = portSamples[columnsToGather[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = portSamples[columnsToGather[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(columnsToGather[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('SYN Flag Count', 2.0006569906769673)
('Flow Duration', 0.029749295421954428)
('Packets Per Second', 0.05486300518772401)
('IAT Total', 0.029749295421954428)
('IAT Max', 0.0008843103101170367)
('IAT Mean', 1.2280702030919074e-05)
('IAT Std', 4.525685204844482e-05)


In [52]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
portDataset['Number of Ports'] = np.random.randint(MinMaxDict['Number of Ports'][0]*0.85, MinMaxDict['Number of Ports'][1]*1.15, NUM_OF_ROWS)

for index, row in portDataset.iterrows():
    i = 0
    for col, factor in zip(columnsToGather[1:], scaling_factors): #iterating over all rows we need to add values to except 'Number of Ports'
        # calculate a random small delta of the factor for adding some randomness
        delta = random.uniform(factor[1] * 0.01, factor[1] * 0.02) # select a dantom

        # apply the randomness to the calculated number
        updatedFactor = factor[1] + random.choice([-1, 1]) * delta

        # calculate the value we want to add into the dataset in the given row
        portDataset.loc[index, col] = row['Number of Ports'] * updatedFactor
        i+=1

In [53]:
# making the SYN Flag Count column have int values instead of floats
portDataset['SYN Flag Count'] = portDataset['SYN Flag Count'].astype(int)

# adding a label to the dataset
portDataset['Label'] = ATTACK_NAME

In [54]:
portDataset

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std,Label
0,971,0,0,0,0,0,0,0,0,0,...,1967,0,0,29.353887,54.215647,28.470199,0.875155,0.012156,0.043229,PortScan
1,504,0,0,0,0,0,0,0,0,0,...,989,0,0,14.797953,27.220348,14.749746,0.453649,0.006268,0.023172,PortScan
2,792,0,0,0,0,0,0,0,0,0,...,1558,0,0,23.230515,43.897096,23.994685,0.687232,0.009857,0.036453,PortScan
3,1106,0,0,0,0,0,0,0,0,0,...,2236,0,0,33.398305,61.857328,33.503827,0.993029,0.013848,0.049445,PortScan
4,1971,0,0,0,0,0,0,0,0,0,...,3893,0,0,57.941667,109.350900,59.592014,1.769278,0.024557,0.090125,PortScan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1508,0,0,0,0,0,0,0,0,0,...,3059,0,0,44.059591,84.201822,44.224728,1.313503,0.018818,0.067031,PortScan
24996,1404,0,0,0,0,0,0,0,0,0,...,2754,0,0,41.068714,78.033099,42.478113,1.219187,0.016986,0.062561,PortScan
24997,1529,0,0,0,0,0,0,0,0,0,...,3016,0,0,46.013254,82.584579,44.673050,1.333085,0.018418,0.068025,PortScan
24998,1640,0,0,0,0,0,0,0,0,0,...,3346,0,0,49.566340,91.325419,48.151390,1.474524,0.019794,0.074998,PortScan


In [55]:
portDataset.describe()

,Number of Ports,Average Packet Size,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,...,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Total,IAT Max,IAT Mean,IAT Std
count,25000.000000,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,25000.0,...,25000.0,25000.000000,25000.0,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,1070.918240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2142.449080,0.0,0.0,31.855464,58.751537,31.863452,0.947050,0.013150,0.048468
std,520.593761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1042.364818,0.0,0.0,15.488802,28.580781,15.501519,0.460757,0.006397,0.023572
min,168.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,330.000000,0.0,0.0,4.928252,9.041702,4.906849,0.145767,0.002025,0.007451
25%,619.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1239.000000,0.0,0.0,18.433287,34.021538,18.436486,0.547850,0.007600,0.028003
50%,1068.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2135.000000,0.0,0.0,31.793312,58.601116,31.759335,0.944319,0.013112,0.048357
75%,1521.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3042.250000,0.0,0.0,45.214843,83.332134,45.240897,1.344008,0.018660,0.068784
max,1971.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4017.000000,0.0,0.0,59.716620,110.123300,59.770917,1.777739,0.024652,0.090971


In [56]:
# save the dataset
portDataset.to_csv('port_scan_dataset_updated_flows.csv', index=False)

---

In [57]:
# # using min-max scaler
# scaler = MinMaxScaler(feature_range=(0, 1))
# df['Scaled'] = scaler.fit_transform(df[['Value']])